In [1]:
import numpy as np
import pandas as pd
import os

In [4]:
x = np.random.randint(5, size=[10,4]) +10
y = np.random.randint(5, size=[10,4])

In [2]:
def pairwiseDist(x, y=None):
    if y is None:
        y = x
    return np.sum((x[:,None]-y)**2,axis=2)**0.5

def arrayDist(x,y):
    return np.sum((x-y)**2,axis=1)**0.5

def errRates(pred, actual):
    return (actual!=pred).sum() / pred.size # return error rate

In [57]:
def initMeans(data, n, algo=1):
    if algo==1: # choose n random points
        idx = np.random.choice(range(data.shape[0]), n, False) # no replace
        out = data[idx,]
    if algo==2: # random means points (0,1)
        out = np.random.random([n,data.shape[1]])
    if algo==3: # always take first n point as centroid
        out = data[:n,]
    return out

def shortestCentroid(centr, mat):
    tmpDist = pairwiseDist(centr,mat) # dist between means and all data pts
    return tmpDist.argmin(axis=0) # find group where distance is smallest

def updateMeans(data, means):
    ## Assign each pt to the mean for which it has the shortest distance
    tmpDist = pairwiseDist(means,data) # dist between means and all data pts
    minDist = tmpDist.argmin(axis=0) # find group where distance is smallest

    ## Calculate new means to be centroid of all the points in the group
    newMeans = np.zeros([len(means),data.shape[1]]) # new mean points
    for n,x in enumerate(means): # loop over all clusters
        tmp = np.vstack( (data[minDist==n,],x) ) # concat data pt and centroid
        newMeans[n] = tmp.mean(axis=0) # new mean = centroid of all pts 
    
    return newMeans,minDist

################################################################################
def kMeans(data, k, trace=False, initAlgo=1):
    means = initMeans(data, k, initAlgo) # initialize mean points
    converged = False
    while not converged:
        newMeans,grpIdx = updateMeans(data, means)
        converged = np.allclose(means,newMeans)
        if trace:
            print(means)
        means = newMeans
        
    return means,grpIdx # return final centroids and labels

In [112]:
updateMeans(x, x[:3])

(array([[13.33333333, 14.        , 14.        , 13.        ],
        [11.        , 12.5       , 10.375     , 11.875     ],
        [10.        , 11.        , 14.        , 12.        ]]),
 array([0, 1, 2, 1, 1, 1, 1, 1, 0, 1], dtype=int64))

In [113]:
z = np.vstack([x,y])

kMeans(z, 2, True, 2)

[[0.37447605 0.12174801 0.34172695 0.38724677]
 [0.29682263 0.21363869 0.96782684 0.33721441]]
[[2.34361901 2.530437   0.08543174 1.59681169]
 [6.73871237 7.95631326 7.38710149 7.6854008 ]]
[[2.34361901 2.530437   0.08543174 1.59681169]
 [6.73871237 7.95631326 7.38710149 7.6854008 ]]
[[ 1.57669264  2.68458518  1.55322107  2.23607379]
 [11.06715567 12.17784666 11.12610014 11.6986728 ]]
[[ 1.57669264  2.68458518  1.55322107  2.23607379]
 [11.06715567 12.17784666 11.12610014 11.6986728 ]]
[[ 1.50697206  2.69859865  1.68665646  2.29418853]
 [11.46065052 12.56162242 11.4660091  12.06351571]]
[[ 1.50697206  2.69859865  1.68665646  2.29418853]
 [11.46065052 12.56162242 11.4660091  12.06351571]]
[[ 1.50063382  2.6998726   1.69878695  2.29947168]
 [11.49642277 12.59651113 11.49690992 12.09668325]]
[[ 1.50063382  2.6998726   1.69878695  2.29947168]
 [11.49642277 12.59651113 11.49690992 12.09668325]]
[[ 1.50005762  2.69998842  1.69988972  2.29995197]
 [11.4996748  12.59968283 11.49971908 12.09969

(array([[ 1.50000048,  2.6999999 ,  1.69999909,  2.2999996 ],
        [11.49999731, 12.59999738, 11.49999768, 12.09999751]]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int64))

In [23]:
def SelectBestFeature(dataMat, selected, Nk, dists):
    # get list of index of currently unselected features
    unselect = np.where(~np.isin(np.arange(dataMat.shape[1]),selected))[0]
    bestCoeff = -1-1e-9 # worst possible coefficient value is -1
    for n,j in enumerate(unselect): # loop over unselected features
        testSet = np.hstack([selected,j]) # add curr feature to selected ones
        means,labels = kMeans(dataMat[:,testSet], Nk) # cluster w/ test features
        coeff = Silhouette(dataMat,labels,dists).mean() # mean silhouette coeff
        #print((coeff,bestCoeff))
        if coeff > bestCoeff: # if this feature produce better coeff
            bestCoeff = coeff # record new best coeff
            outs = (j,coeff,means,labels) # record output variables
    #print(unselect)
    return outs # output: the feature, best coeff, means, and labels
################################################################################

def ForwardSelect(data, k):
    selected = np.zeros(0, int) # idx of selected features, start w/ empty
    baseCoeff = -1-1e-9 # -1 is worst possible performance
    dM = pairwiseDist(data) # pre-calc distance matrix for memoization
    
    converged = False    
    while not converged: # loop until convergence
        bestFeat,bestCoeff,means,labels = SelectBestFeature(data,selected,k,dM) 
        if bestCoeff <= baseCoeff: # if new feature doesn't improve performance
            converged = True
        else: # if new feature improves performance
            print(bestCoeff-baseCoeff)
            selected = np.hstack([selected,bestFeat]) # add feature to selection
            baseCoeff = bestCoeff # set new coeff as baseline performance
            outs = (means,labels) # save output vars
            if len(selected) == data.shape[1]: 
                converged = True # algo converged if all features selected
    return (selected,)+outs # return selected features, means, cluster labels

In [336]:
testData = np.random.randint(0,10,[100,8])

ForwardSelect(testData,2)

1.1044337798534718
0.0027974021309899316


(array([3, 4], dtype=int64), array([[2.03225794, 4.59677437],
        [7.39473531, 3.02631531]]), array([1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int64))

In [24]:
################################################################################
def Silhouette(data, labels, distMat=None):
    if distMat is None:
        distMat = pairwiseDist(data) # calc pairwise dist if not provided
    grpIdx = pd.Series(labels).groupby(labels).groups.items() # idx for each grp
    
    aVals = np.zeros(data.shape[0]) # pre-allocate a and b-values for data
    bVals = np.zeros(data.shape[0])
    for grp,idx in grpIdx: # loop over all groups
        aVals[idx] = distMat[np.ix_(idx,idx)].mean(axis=1) # a's for curr grp
        
        # loop over all groups that's not the current gruop
        tmp = np.zeros([len(grpIdx)-1,len(idx)]) # tmp for all b's for curr grp
        for n,(_,outIdx) in enumerate([x for x in grpIdx if x[0]!=grp]):
            # calculate mean dist of points within cluster to out of cluster
            tmp[n,] = distMat[np.ix_(idx,outIdx)].mean(axis=1) 
        #print(tmp.shape)
        #print(idx.shape)
        bVals[idx] = tmp.min(axis=0) # pick min b of all out-groups

    return (bVals-aVals)/np.maximum(aVals,bVals) # return silhouette coeff

In [422]:
pairwiseDist(spamMat[]

1.218264548496111
0.10202167454565866
0.011917492844527922
0.0915066380507315
0.010737041792362234
0.013112969092185567


(array([5, 4, 7, 6, 3, 2], dtype=int64),
 array([[2.44444449e-01, 7.25177778e+01, 1.74999996e-01, 1.25800000e+01,
         1.30888889e+00, 1.69444481e-01],
        [1.65925926e-01, 7.17748148e+01, 2.96296296e-02, 9.58481481e+00,
         8.69259259e-01, 3.69074074e+00],
        [3.33999997e-01, 7.21999999e+01, 1.96666675e-01, 9.68799987e+00,
         1.65866667e+00, 1.97333337e+00],
        [5.66260163e-01, 7.28155285e+01, 5.20325203e-03, 8.38772358e+00,
         1.37284553e+00, 3.50658537e+00],
        [3.46399994e+00, 7.10260000e+01, 1.00399998e+00, 6.19600003e+00,
         2.71799998e+00, 1.82800003e+00],
        [2.11923077e-01, 7.34469231e+01, 9.57692308e-01, 8.59884615e+00,
         2.10961538e+00, 6.69230771e-02]]),
 array([1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 1,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 3, 3, 3, 1,
        3, 1, 3, 1, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [25]:
irisFile = os.path.join('./data/', 'iris.data')
irisName = ['sepalLen', 'sepalWth', 'petalLen', 'petalWth', 'class']
raw = pd.read_csv(irisFile , names=irisName)  # read CSV file
irisFeats = irisName[:-1]
irisMat = raw[irisFeats].values
irisK = len(raw['class'].unique())

In [32]:
ForwardSelect(irisMat, irisK)

1.5547284946256994
0.002021954381113966


(array([2, 1], dtype=int64), array([[1.464     , 3.418     ],
        [4.40000004, 2.75396827],
        [5.76756793, 3.07297301]]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
        2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
        2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2], dtype=int64))

In [33]:
glassData = os.path.join('./data/', 'glass.data')
glassNames = ['id','RI','Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'class']
raw = pd.read_csv(glassData , names=glassNames)  # read CSV file

glassFeats = glassNames[1:-1] # list of feature names
glassMat = raw[glassFeats].values # 2d-array of feature values
glassK = len(raw['class'].unique()) # number of classes

In [35]:
ForwardSelect(glassMat, glassK)

1.280279874484533
0.08921012138212236
0.009817673937100979


(array([5, 3, 2], dtype=int64), array([[0.11730769, 0.80269231, 3.73115385],
        [0.65064516, 2.21      , 0.        ],
        [0.39541667, 1.51833333, 2.28708333],
        [0.5746087 , 1.38417391, 3.54991304],
        [0.11285714, 1.04285714, 0.15142857],
        [1.49999787, 2.40249904, 3.05749732]]), array([0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
        3, 3, 3, 0, 0, 3, 0, 3, 2, 2, 2, 2, 3, 3, 3, 3, 0, 3, 0, 0, 0, 0,
        0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 0, 2, 1, 1, 4, 4, 4,
        4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 4, 4,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 0, 3,
        3, 3, 3, 0, 3, 3, 3, 0, 0, 5, 2, 2, 2, 1, 1, 1, 4, 1, 1, 4, 2, 4,
        2, 2, 2, 2, 2, 4, 1, 4, 4, 5, 5, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 

In [36]:
spamData = os.path.join('./data/', 'spambase.data')
spamNames = ['make', 'address', 'all', '3d', 'our', 'over', 'remove',
	'internet', 'order', 'mail', 'receive', 'will', 'people', 'report',
	'addresses', 'free', 'business', 'email', 'you', 'credit', 'your', 'font',
	'0', 'money', 'hp', 'hpl', 'george', '650', 'lab', 'labs', 'telnet', '857',
	'data', '415', '85', 'technology', '1999', 'parts', 'pm', 'direct', 'cs',
	'meeting', 'original', 'project', 're', 'edu', 'table', 'conference',
	'semicolon', 'paren', 'bracket', 'exclaim', 'dollar', 'pound', 'capsAvg',
	'capsMax', 'capsTotal', 'class']
raw = pd.read_csv(spamData , names=spamNames)  # read CSV file

spamFeats = spamNames[:-1] # list of feature names
spamMat = raw[spamFeats].values # 2d-array of feature values
spamK = len(raw['class'].unique()) # number of classes

In [38]:
ForwardSelect(spamMat, spamK) # run algorithm

1.842031262105331
0.02755507341246377


(array([56, 20], dtype=int64), array([[2.82540152e+03, 8.86363638e-01],
        [2.08203401e+02, 8.07498322e-01]]), array([1, 1, 0, ..., 1, 1, 1], dtype=int64))

In [39]:
spamMat.shape

(4601, 57)

In [64]:
################################################################################
def evalFitness(dataMat, k, pop, preEval, dist): # eval fitness of individuals
    fitness = np.empty(pop.shape[0]) # store fitness of individuals
    for n,indv in enumerate(pop): # loop over populations one by one
        gene = ''.join(['1' if x else '0' for x in indv]) # string repr of DNA
        if gene in preEval: # combo of features previously evaluated
            fitness[n] = preEval[gene] # recall from dict
        else: # never evaluated before
            means,labels = kMeans(dataMat[:,indv], k) # cluster w/ features
            fitness[n] = Silhouette(dataMat,labels,dist).mean()+1 # fit > 0
            preEval[gene] = fitness[n] # store into dict for memoization
    return fitness,preEval

def crossOver(pop, parentIdx):
    idxDad = parentIdx[:len(parentIdx)//2] # first half of selected
    idxMom = parentIdx[len(parentIdx)//2:] # second half of selected
    
    breakPts = np.random.randint(1,pop.shape[1],pop.shape[0]//2) # x-over points
    
    out = np.empty(pop.shape, bool) # pre-allocate array for next gen
    for n,(d,m) in enumerate(zip(idxDad,idxMom)):
        out[n] = np.hstack([ pop[d,:breakPts[n]],pop[m,breakPts[n]:] ])
        out[n+len(pop)] = np.hstack([ pop[m,:breakPts[n]],pop[d,breakPts[n]:] ])
    return out

def selectParents(fitness, popSize):
    cumFit = np.cumsum(fitness)/np.sum(fitness) # cum array of normalized fitness
    rands = np.random.rand(popSize) # uniform random between 0,1
    outInd = np.searchsorted(cumFit, rands) # higher prob of select high fitness
    return outInd

def mutate(pop, prob):
    toMutate = np.where(np.random.rand(pop.shape[0])<prob) # idx of pop to mutate
    mutatePts = np.random.randint(0,pop.shape[1],len(toMutate)) # where to mutate
    for idx,n in zip(toMutate,mutatePts): # mutate selected individuals
        pop[idx,n] = ~pop[idx,n] # flip the selection bit
    return pop

def geneticAlgoSelect(data, k, prm):
    pop = np.random.rand(prm['popSize'],data.shape[1]) > 0.5 # random init pop
    memo = dict() # dict of result for memoization
    dMat = pairwiseDist(data) # pre-calc distance matrix for memoization
    
    baseFitness = 0 # worst possible fitenss score
    converged = False
    while not converged: # loop until GA has converged
        fit,memo = evalFitness(data, k, pop, memo, dMat) # evaluate fitness
        bestIdx = np.argmax(fit) # keep track of best indiviaul
        bestFit,bestIndv = fit[bestIdx],pop[bestIdx] # best fit and features
        
        if bestFit-baseFitness < prm['minImprove']: # min improvement required
            converged = True
            out = bestFit-1,np.where(bestIndv) # silhouette coeff and list
        else: # not converged, selection + crossover + mutation
            parentInd = selectParents(fit, pop.shape[0]) # select parents
            pop = crossOver(pop, parentInd) # cross-over to get next gen
            pop = mutate(pop,prm['mutateProb'])
    return out

In [65]:
spamPrm = {'popSize':100, 'minImprove':0.02, 'mutateProb':0.05}
geneticAlgoSelect(spamMat, spamK, spamPrm)

IndexError: index 100 is out of bounds for axis 0 with size 100

In [47]:
np.random.rand(100,57) > 0.5

array([[False, False,  True, ..., False,  True,  True],
       [False,  True, False, ...,  True, False,  True],
       [ True, False,  True, ..., False,  True,  True],
       ...,
       [False, False, False, ...,  True, False,  True],
       [ True, False, False, ..., False, False, False],
       [False,  True,  True, ..., False, False, False]])

In [40]:
testX = np.array([[1,1,1],[10,10,10],[11,11,11],[11,10,11],
                  [2,2,1],[1.5,1.5,1.5],[12,12,12]]) + np.random.random([7,3])/10
testY = np.array([1,2,2,2,1,1,2])
testY_bad = np.array([2,2,1,2,1,2,1])
print(testX)

print(Silhouette(testX,testY))
print(Silhouette(testX,testY_bad))

[[ 1.04915121  1.07765959  1.04720735]
 [10.09038171 10.01132724 10.05882021]
 [11.05405553 11.03845812 11.07578113]
 [11.08976924 10.03406745 11.01091978]
 [ 2.04331281  2.00951885  1.06552362]
 [ 1.59479717  1.51716246  1.55381817]
 [12.07416047 12.05564935 12.02936141]]
[0.95676181 0.88977003 0.93290523 0.92456144 0.95460956 0.96544999
 0.89585935]
[ 0.34018504 -0.1717065   0.34488496 -0.24973533 -0.29011733  0.33463491
  0.39248036]


## Sources

- [A Modified k-means Algorithm to Avoid Empty Clusters](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.3148&rep=rep1&type=pdf)